It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?




    df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
                        'londON_StockhOlm',

                        'Budapest_PaRis', 'Brussels_londOn'],
                        'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                            'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                        'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
                        '12. Air France', '"Swiss Air"']})
     df.head()

1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each

second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [66]:
import pandas as pd
import numpy as np

In [67]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
                        'londON_StockhOlm',

                        'Budapest_PaRis', 'Brussels_londOn'],
                        'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                            'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                        'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
                        '12. Air France', '"Swiss Air"']})
df.head()
TempData = df.copy()

### Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column aninteger column (instead of a float column).

In [68]:
res =  list(np.where(df["FlightNumber"].isnull()))
for x in res[0]:
    df.loc[x,"FlightNumber"] = df.loc[x-1,"FlightNumber"]+10
df["FlightNumber"] = df["FlightNumber"].astype(int)
df.dtypes

From_To         object
FlightNumber     int32
RecentDelays    object
Airline         object
dtype: object

### The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [69]:
TempData[['Source','Destination']] = df.From_To.str.split("_",expand=True)
TempData.drop(["From_To","FlightNumber","RecentDelays","Airline"],axis=1,inplace=True)
TempData

,Source,Destination
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


### Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [70]:
def convertCase(x):
    st=x[1:].lower()
    x = x[0].upper() 
    return x+st
TempData['Source'] = TempData['Source'].apply(convertCase)
TempData['Destination'] = TempData['Destination'].apply(convertCase)
TempData

,Source,Destination
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


### Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [72]:
df.drop("From_To",axis=1,inplace=True)
df = pd.concat([df,TempData],axis=1)
df

,FlightNumber,RecentDelays,Airline,Source,Destination
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


### In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, eachsecond value in its own column, and so on. If there isn't an Nth value, the value should be NaN. Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [104]:
d2 = (pd.DataFrame(df['RecentDelays'].values.tolist(), index=df.index)
        .rename(columns = lambda x: 'Delay{}'.format(x+1)))

df = df.join(d2)

In [112]:
columnsTitle =["Source","Destination","FlightNumber","Airline","Delay1","Delay2","Delay3"]
df = df.reindex(columns=columnsTitle)
df

,Source,Destination,FlightNumber,Airline,Delay1,Delay2,Delay3
0,London,Paris,10045,KLM(!),23.0,47.0,NaN
1,Madrid,Milan,10055,<Air France> (12),NaN,NaN,NaN
2,London,Stockholm,10065,(British Airways. ),24.0,43.0,87.0
3,Budapest,Paris,10075,12. Air France,13.0,NaN,NaN
4,Brussels,London,10085,"""Swiss Air""",67.0,32.0,NaN
